## Configuration et imports

In [1]:
import requests
import pandas as pd
import numpy as np
import yfinance as yf
import json

# Configuration robuste pour éviter les blocages SEC
HEADERS = {
    "User-Agent": "niels.wittoeck@gmail.com",
    "Accept-Encoding": "gzip, deflate",
}

session = requests.Session()
session.headers.update(HEADERS)

## Fonction de conversion de ticker en CIK sur la SEC

In [2]:
def get_cik(ticker):
    url = "https://www.sec.gov/files/company_tickers.json"
    try:
        resp = session.get(url)
        resp.raise_for_status()
        data = resp.json()
        target = ticker.upper().strip()
        for item in data.values():
            if item["ticker"] == target:
                return str(item["cik_str"]).zfill(10)
    except Exception as e:
        print(f"Erreur de connexion : {e}")
    return None

## Extraction des données

In [3]:
def extract_history(facts, tags_priority):
    history = {}
    for tag in tags_priority:
        if tag in facts:
            units = facts[tag].get("units", {}).get("USD", [])
            for d in sorted(units, key=lambda x: x.get('end', '')):
                year = d["end"][:4]
                val = d["val"]
                if d.get("form") == "10-K": # Priorité aux rapports annuels audités
                    history[year] = val
    return history

## Choix des hypothèses prises en compte pour le DCF

### Fonctions d'automatisation du choix des hypothèses pour le DCF

In [4]:
def estimate_wacc(ticker):
    ticker = yf.Ticker(ticker)
    
    # 1. Récupération du Bêta
    beta = ticker.info.get('beta', 1.1)  # 1.1 par défaut si non trouvé
    
    # 2. Récupération du Taux sans risque (Risk-Free Rate) 
    # On prend le rendement actuel des US Treasury 10Y (^TNX)
    tnx = yf.Ticker("^TNX")
    hist_tnx = tnx.history(period="1d")
    rf = hist_tnx['Close'].iloc[-1] / 100 if not hist_tnx.empty else 0.035
    
    # 3. Prime de risque du marché (Equity Risk Premium)
    # Historiquement entre 4.5% et 6%
    erp = 0.055 
    
    wacc = rf + (beta * erp)
    
    return wacc

Cette fonction calcule le taux d'actualisation en combinant trois données de marché essentielles : elle récupère d'abord le Bêta pour mesurer la volatilité spécifique de l'action par rapport au marché, puis elle extrait en temps réel le rendement des obligations d'État à 10 ans (Taux sans risque) via le symbole ^TNX. En y ajoutant une Prime de risque de marché de 5,5 %, elle applique la formule du MEDAF pour transformer ces indicateurs en un taux de rendement exigé. Concrètement, elle permet de définir un coût du capital dynamique qui s'ajuste automatiquement à la dangerosité de l'entreprise et aux conditions économiques actuelles.

In [5]:
def estimate_growth(df):
    latest_yr = df.columns[-1]
    
    # ROIC = EBIT / (Capitaux Propres + Dette Totale)
    ebit = df.loc["EBIT", latest_yr]
    equity = df.loc["Stockholders Equity", latest_yr]
    debt = df.loc["Total Debt", latest_yr] if "Total Debt" in df.index else 0
    
    roic = ebit / (equity + debt)
    
    # Taux de réinvestissement = CapEx / Operating Cash Flow
    capex = abs(df.loc["CapEx", latest_yr])
    ocf = df.loc["OCF", latest_yr]
    
    reinvest_rate = min(capex / ocf, 0.8) if ocf > 0 else 0.2
    
    # g = ROIC * Taux de réinvestissement
    growth_fcf = roic * reinvest_rate
    
    return max(min(growth_fcf, 0.20), 0.02) # Capé entre 2% et 20% pour la stabilité

Cette fonction estime la croissance future des flux de trésorerie en s'appuyant sur la performance opérationnelle réelle de l'entreprise. Elle calcule d'abord le ROIC (Retour sur Capital Investi) pour mesurer l'efficacité avec laquelle l'entreprise génère des profits à partir de ses ressources, puis elle détermine le Taux de réinvestissement en comparant les dépenses d'investissement (CapEx) au flux de trésorerie opérationnel. En multipliant ces deux indicateurs, la fonction déduit un taux de croissance logique : plus une entreprise est rentable et réinvestit une part importante de ses gains, plus sa capacité de croissance projetée est élevée.

### Choix du mode de sélection des hypothèses

In [6]:
def mode(df, ticker):
    choice = ""
    # On boucle tant que le choix n'est pas dans la liste des options valides
    while choice not in ["1", "2"]:
        choice = input("Choix des hypothèses : automatique (1) ou manuel (2)? ").strip()

    latest_yr = df.columns[-1]
    last_revenue = df.loc["Revenue", latest_yr]
    last_fcf = df.loc["Free Cash Flow", latest_yr]
    wacc = estimate_wacc(ticker)

    # Initialisation des valeurs par défaut pour éviter les erreurs
    growth_fcf_auto = 0.0
    custom_revenue_growth = 0.0
    custom_ebit_margin = 0.0
    custom_fcf_conversion = 0.0

    if choice == "2":
        print("\n--- Saisie des hypothèses manuelles ---")
        try:
            g_rev_input = float(input("Croissance annuelle du CA (%) : "))
            margin_input = float(input("Marge Opérationnelle cible (%) : "))
            conv_input = float(input("Conversion FCF (% de l'EBIT) : "))
            
            custom_revenue_growth = g_rev_input / 100
            custom_ebit_margin = margin_input / 100
            custom_fcf_conversion = conv_input / 100
            
            mode_calcul = "manuel"
            scenario_params = {
                "Croissance CA": f"{g_rev_input}%",
                "Marge EBIT": f"{margin_input}%",
                "Conv. FCF": f"{conv_input}%"
            }
            return mode_calcul, wacc, latest_yr, last_revenue, last_fcf, growth_fcf_auto, custom_revenue_growth, custom_ebit_margin, custom_fcf_conversion, scenario_params
        except ValueError:
            print("Erreur de saisie. Passage en mode automatique.")
    
    # Mode Auto par défaut
    mode_calcul = "auto"
    growth_fcf_auto = estimate_growth(df) # Assure-toi que le nom de la fonction correspond
    print(f"\nMode Auto activé. Croissance FCF estimée : {growth_fcf_auto*100:.2f}%")
    scenario_params = {"Croissance FCF (Auto)": f"{growth_fcf_auto*100:.2f}%"}
    
    return mode_calcul, wacc, latest_yr, last_revenue, last_fcf, growth_fcf_auto, custom_revenue_growth, custom_ebit_margin, custom_fcf_conversion, scenario_params

## Calcul du DCF

In [7]:
def DCF(df, ticker, mode_calcul, wacc, latest_yr, last_revenue, last_fcf, growth_fcf_auto, custom_revenue_growth, custom_ebit_margin, custom_fcf_conversion):
    print(f"\n--- CALCUL DU DCF ({ticker}) ---")
    print(f"WACC utilisé : {wacc*100:.2f}%")

    projection_years = 5
    terminal_growth = 0.02 
    projected_fcfs = []
    discounted_fcfs = []

    current_revenue = last_revenue

    for i in range(1, projection_years + 1):
        if mode_calcul == "manuel":
            current_revenue *= (1 + custom_revenue_growth)
            projected_ebit = current_revenue * custom_ebit_margin
            fcf_proj = projected_ebit * custom_fcf_conversion
        else:
            base_fcf = last_fcf if i == 1 else projected_fcfs[-1]
            fcf_proj = base_fcf * (1 + growth_fcf_auto)

        fcf_disc = fcf_proj / ((1 + wacc) ** i)
        projected_fcfs.append(fcf_proj)
        discounted_fcfs.append(fcf_disc)
        print(f"Année {i} : FCF Projeté = {fcf_proj/1e6:,.0f} M$ | Actualisé = {fcf_disc/1e6:,.0f} M$")

    tv = (projected_fcfs[-1] * (1 + terminal_growth)) / (wacc - terminal_growth)
    tv_disc = tv / ((1 + wacc) ** projection_years)
    
    enterprise_value = sum(discounted_fcfs) + tv_disc

    total_debt = df.loc["Total Debt", latest_yr] if "Total Debt" in df.index else 0
    cash = df.loc["Cash", latest_yr] if "Cash" in df.index else 0
    
    net_debt = np.nan_to_num(total_debt) - np.nan_to_num(cash)
    
    equity_value = enterprise_value - net_debt

    yf_ticker = yf.Ticker(ticker)
    shares = yf_ticker.info.get("sharesOutstanding", 1)
    current_price = yf_ticker.info.get("currentPrice", 1)

    intrinsic_value = equity_value / shares
    upside = (intrinsic_value / current_price - 1) * 100
    
    return intrinsic_value, current_price, upside, enterprise_value, net_debt, equity_value, shares

## Exécution du code

In [8]:
ticker = input("Entrez le ticker (ex: AAPL, TSLA) : ").upper().strip()
cik = get_cik(ticker)

if cik:
    print(f"Ticker {ticker} trouvé (CIK: {cik})")

    # 1. Extraction des données de la SEC
    url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
    print(url)
    facts = session.get(url).json().get("facts", {}).get("us-gaap", {})

    config = {
        "Revenue": ["RevenueFromContractWithCustomerExcludingAssessedTax", "SalesRevenueNet", "Revenues", "SalesRevenueGoodsNet"],
        "EBIT": ["OperatingIncomeLoss"],
        "Net Income": ["NetIncomeLoss", "ProfitLoss"],
        "CapEx": ["PaymentsToAcquirePropertyPlantAndEquipment", "NetCashProvidedByUsedInInvestingActivities"],
        "OCF": ["NetCashProvidedByUsedInOperatingActivities"],
        "Stockholders Equity": ["StockholdersEquity"],
        "Total Debt": ["LongTermDebt", "LongTermDebtNoncurrent", "DebtCurrent"],
        "Cash": ["CashAndCashEquivalentsAtCarryingValue", "ShortTermInvestments", "AvailableForSaleSecuritiesCurrent"]
    }

    raw_results = {label: extract_history(facts, tags) for label, tags in config.items()}
    df = pd.DataFrame(raw_results).T
    df = df.reindex(sorted(df.columns), axis=1).iloc[:, -7:] # 7 dernières années

    # 2. Calculs préliminaires
    df.loc["Free Cash Flow"] = df.loc["OCF"] - df.loc["CapEx"]
    df = df.dropna(axis=1, how='all')

    # 3. Affichage de l'historique financier
    print(f"\n--- HISTORIQUE FINANCIER : {ticker} ---")
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    df_display = df.map(lambda x: f"{x/1e6:,.0f} M$" if pd.notnull(x) else "-")
    display(df_display)

    # 4. CHOIX DES HYPOTHÈSES : Capture des variables retournées par mode()
    # On déballe le tuple retourné par la fonction pour définir wacc, mode_calcul, etc.
    (mode_calcul, wacc, latest_yr, last_revenue, last_fcf, 
     growth_fcf_auto, custom_rev, custom_margin, custom_conv, scenario_params) = mode(df, ticker)

    # 5. CALCUL DU DCF : Passage des variables capturées à la fonction DCF
    (intrinsic_value, current_price, upside, 
     enterprise_value, net_debt, equity_value, shares) = DCF(
        df, ticker, mode_calcul, wacc, latest_yr, last_revenue, last_fcf, 
        growth_fcf_auto, custom_rev, custom_margin, custom_conv
    )

    # 6. Récupération des détails pour l'affichage final
    total_debt = np.nan_to_num(df.loc["Total Debt", latest_yr]) if "Total Debt" in df.index else 0
    cash = np.nan_to_num(df.loc["Cash", latest_yr]) if "Cash" in df.index else 0

    # 7. AFFICHAGE DES RÉSULTATS FINAUX
    print("\n" + "="*40)
    print(f"RÉSULTATS DE LA VALORISATION ({ticker})")
    print("="*40)
    print(f"Hypothèses utilisées : {scenario_params}")
    print("-" * 40)
    print(f"Valeur d'Entreprise (EV)  : {enterprise_value/1e9:,.2f} Md$")
    print(f"  - Dette Nette           : {net_debt/1e9:,.2f} Md$ (Dette: {total_debt/1e9:.1f} - Cash: {cash/1e9:.1f})")
    print(f"Valeur des Capitaux (EqV) : {equity_value/1e9:,.2f} Md$")
    print(f"Nombre d'actions          : {shares/1e6:,.0f} M")
    print("-" * 40)
    print(f"VALEUR INTRINSÈQUE / ACTION : {intrinsic_value:.2f} $")
    print(f"PRIX ACTUEL                 : {current_price:.2f} $")

    if intrinsic_value > current_price:
        print(f"SOUS-ÉVALUÉ (Potentiel : +{upside:.1f}%)")
    else:
        print(f"SUR-ÉVALUÉ (Potentiel : {upside:.1f}%)")
else:
    print("Impossible de continuer sans le CIK.")

Ticker AMZN trouvé (CIK: 0001018724)
https://data.sec.gov/api/xbrl/companyfacts/CIK0001018724.json

--- HISTORIQUE FINANCIER : AMZN ---


,2018,2019,2020,2021,2022,2023,2024
Revenue,"72,383 M$","87,437 M$","125,555 M$","469,822 M$","513,983 M$","574,785 M$","637,959 M$"
EBIT,"3,786 M$","3,879 M$","6,873 M$","24,879 M$","12,248 M$","36,852 M$","68,593 M$"
Net Income,"3,027 M$","3,268 M$","7,222 M$","33,364 M$","-2,722 M$","30,425 M$","59,248 M$"
CapEx,"-12,369 M$","-24,281 M$","-59,611 M$","-58,154 M$","-37,601 M$","-49,833 M$","-94,342 M$"
OCF,"30,723 M$","38,514 M$","66,064 M$","46,327 M$","46,752 M$","84,946 M$","115,877 M$"
Stockholders Equity,"43,549 M$","62,060 M$","93,404 M$","138,245 M$","146,043 M$","201,875 M$","285,970 M$"
Total Debt,"23,495 M$","23,414 M$","31,816 M$","48,744 M$","67,150 M$","58,314 M$","52,623 M$"
Cash,"31,750 M$","36,092 M$","42,122 M$","36,220 M$","53,888 M$","73,387 M$","78,779 M$"
Free Cash Flow,"43,092 M$","62,795 M$","125,675 M$","104,481 M$","84,353 M$","134,779 M$","210,219 M$"



Mode Auto activé. Croissance FCF estimée : 16.21%

--- CALCUL DU DCF (AMZN) ---
WACC utilisé : 11.71%
Année 1 : FCF Projeté = 244,288 M$ | Actualisé = 218,683 M$
Année 2 : FCF Projeté = 283,879 M$ | Actualisé = 227,487 M$
Année 3 : FCF Projeté = 329,886 M$ | Actualisé = 236,646 M$
Année 4 : FCF Projeté = 383,350 M$ | Actualisé = 246,174 M$
Année 5 : FCF Projeté = 445,478 M$ | Actualisé = 256,086 M$

RÉSULTATS DE LA VALORISATION (AMZN)
Hypothèses utilisées : {'Croissance FCF (Auto)': '16.21%'}
----------------------------------------
Valeur d'Entreprise (EV)  : 3,875.44 Md$
  - Dette Nette           : -26.16 Md$ (Dette: 52.6 - Cash: 78.8)
Valeur des Capitaux (EqV) : 3,901.59 Md$
Nombre d'actions          : 10,690 M
----------------------------------------
VALEUR INTRINSÈQUE / ACTION : 364.97 $
PRIX ACTUEL                 : 230.82 $
SOUS-ÉVALUÉ (Potentiel : +58.1%)
